In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
from tqdm import tqdm

In [78]:
ds_disk = xr.open_dataset("netCDF/HAMD.nc")

In [79]:
ds_disk

<xarray.Dataset>
Dimensions:  (date: 470, time(s): 288, error: 3, agency: 3)
Coordinates:
  * date     (date) datetime64[ns] 2023-01-01 2023-01-02 ... 2024-04-15
  * time(s)  (time(s)) float64 300.0 600.0 900.0 ... 8.58e+04 8.61e+04 8.64e+04
  * error    (error) object 'north' 'east' 'up'
  * agency   (agency) object 'igs' 'ckm' 'nic'
Data variables:
    HAMD     (agency, error, date, time(s)) float64 ...

In [80]:
my_df = ds_disk.to_dataframe()

In [81]:
my_df = my_df.reset_index()

In [82]:
my_df['minute'] = my_df['time(s)']/60 % 60
my_df['hour'] = (my_df['time(s)']/60/60).apply(lambda x: np.floor(x))
my_df['full_date'] = my_df.apply(lambda x: x['date'] + timedelta(hours=x['hour'], minutes=x['minute']), axis=1)
cord_dict = {'north':'Lat', 'east':'Lon', 'up':'Alt'}
my_df['error'] = my_df['error'].apply(lambda x: cord_dict[x])
my_df = my_df[['full_date', 'error', 'HAMD', 'agency']]

In [83]:
my_df

,full_date,error,HAMD,agency
0,2023-01-01 00:05:00,Lat,NaN,igs
1,2023-01-01 00:05:00,Lat,NaN,ckm
2,2023-01-01 00:05:00,Lat,NaN,nic
3,2023-01-01 00:05:00,Lon,NaN,igs
4,2023-01-01 00:05:00,Lon,NaN,ckm
...,...,...,...,...
1218235,2024-04-16 00:00:00,Lon,NaN,ckm
1218236,2024-04-16 00:00:00,Lon,NaN,nic
1218237,2024-04-16 00:00:00,Alt,NaN,igs
1218238,2024-04-16 00:00:00,Alt,NaN,ckm


In [84]:
station_name = my_df.columns[2]

In [85]:
pivot_df = my_df.pivot_table(index=['full_date', 'agency'], columns='error', values='HAMD', aggfunc='mean')
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df.fillna(1000)

In [86]:
pivot_df.columns = ['Time Utc', 'Correction_flag', 'Alt', 'Lat', 'Lon']

pivot_df['station_name'] = station_name

pivot_df = pivot_df[['Time Utc', 'station_name', 'Lat', 'Lon', 'Alt', 'Correction_flag']]

ValueError: Length mismatch: Expected axis has 2 elements, new values have 5 elements

In [ ]:
pivot_df

In [ ]:
pivot_df.to_excel("HAMD.xlsx")

In [15]:
def do_all(name):
    ds_disk = xr.open_dataset("netCDF/" + name + ".nc")
    my_df = ds_disk.to_dataframe()
    my_df = my_df.reset_index()
    my_df['minute'] = my_df['time(s)']/60 % 60
    my_df['hour'] = (my_df['time(s)']/60/60).apply(lambda x: np.floor(x))
    my_df['full_date'] = my_df.apply(lambda x: x['date'] + timedelta(hours=x['hour'], minutes=x['minute']), axis=1)
    cord_dict = {'north':'Lat', 'east':'Lon', 'up':'Alt'}
    my_df['error'] = my_df['error'].apply(lambda x: cord_dict[x])
    my_df = my_df[['full_date', 'error', name, 'agency']]
    station_name = my_df.columns[2]
    pivot_df = my_df.pivot_table(index=['full_date', 'agency'], columns='error', values=name, aggfunc='mean')
    pivot_df.reset_index(inplace=True)
    pivot_df = pivot_df.fillna(1000)
    pivot_df.columns = ['Time Utc', 'Correction_flag', 'Alt', 'Lat', 'Lon']

    pivot_df['station_name'] = station_name

    pivot_df = pivot_df[['Time Utc', 'station_name', 'Lat', 'Lon', 'Alt', 'Correction_flag']]
    pivot_df.to_excel(name + ".xlsx")

In [16]:
my_list = os.listdir(r"C:\Users\BarakDav\new_simulation\netCDF")
my_list = [i for i in my_list if ".nc" in i]

In [33]:
for name in tqdm(my_list):
    do_all(name[:-3])

 44%|█████████████████████████████████████▎                                              | 4/9 [03:09<03:56, 47.36s/it]


ValueError: Length mismatch: Expected axis has 2 elements, new values have 5 elements

In [17]:
my_list

['ARUC.nc',
 'BHR3.nc',
 'BHR4.nc',
 'DRAG.nc',
 'HAMD.nc',
 'KITG.nc',
 'KRS1.nc',
 'RAMO.nc',
 'TEHN.nc']

In [18]:
for name in tqdm(my_list[6:]):
    do_all(name[:-3])

  0%|                                                                                            | 0/3 [00:34<?, ?it/s]


ValueError: Length mismatch: Expected axis has 2 elements, new values have 5 elements

In [36]:
my_list[5:]

['KITG.nc', 'KRS1.nc', 'RAMO.nc', 'TEHN.nc']

In [ ]:
#hamd not working need to check it